## The script below creates a Supply Chain BOM based on Enginerring BOM using data from a out-dated supply chain BOM. It also enables the supply chain BOM to sync with the Engineering BOM. Developed by yifei.li@byton.com

In [1]:
import pandas as pd
import numpy as np
import xlrd, openpyxl
import csv, datetime

## Load EBOM, Chi's Supply Chain BOM and Shaolong's BOM

In [2]:
#Engineering BOM (EBOM). Updated weekly and has the most up-to-date Part Number and Revision. 
#Supply Chain BOM is based on EBOM with additional sourcing and supplier quality information.

EBOM = pd.read_excel("/home/yifei/Documents/BOM_tool/EBOM 8.23.18.xlsx")
# new_header = EBOM.iloc[0]
# EBOM = EBOM[1:]
# EBOM.columns = new_header
EBOM.columns = EBOM.columns.str.replace("\(R\)\ ", "") # trim (R) away from the header
print("Engineering VP BOM Shape: ",EBOM.shape)
EBOM.head(5)
# print(EBOM.columns.values)
# EBOM.shape





Engineering VP BOM Shape:  (3164, 14)


,Identifier,Title,Revision,Description,QTY,UOM,Purchased Part Type,Maturity,Part Type,System,SubSystem,Legacy Part Number,Legacy Part Revision,Configuration
0,0,AA-ZZ000001NN-03,A.1,PHTM EC001 VP,1,EA EACH,NaN,In Work,Phantom,Z Vehicle Top Level Cfg,Z Z NA,NaN,NaN,NaN
1,1,AA-AZ000002NN-03,A.1,PHTM BIW VP,1,EA EACH,NaN,In Work,Phantom,A BIW,A Z NA,NaN,NaN,NaN
2,1|1,AA-AF000002NN-03,A.1,PHTM BIW ROOF STRUCTURE VP,1,EA EACH,NaN,In Work,Phantom,A BIW,A F Roof Structure,NaN,NaN,NaN
3,1|1|1,AA-AF003414NN-00,A.2,FULL_ROOF,1,EA EACH,NaN,In Work,NaN,A BIW,A F Roof Structure,EC001_02_521_000,NaN,NaN
4,1|1|1|1,AA-AF003415NN-00,A.1,FULL ROOF,1,NaN,NaN,In Work,NaN,A BIW,A F Roof Structure,EC001_02_501_101_PRT_019,19,NaN


In [3]:
print(EBOM.columns.values)

['Identifier' 'Title' 'Revision' 'Description' 'QTY' 'UOM'
 'Purchased Part Type' 'Maturity' 'Part Type' 'System' 'SubSystem'
 'Legacy Part Number' 'Legacy Part Revision' 'Configuration']


Removing Phantom part and Fasterners and filter only "Buy" part leaves us 1461 Unique PNs

In [4]:
EBOM_filtered = EBOM[(EBOM["Purchased Part Type"]=="Buy") & (EBOM["Part Type"]!="Phantom") & (EBOM["System"]!="Y Fasteners")]
print(EBOM_filtered.shape)


(1461, 14)


In [5]:
#Chi's SCBOM. We need the column names from Chi's SCBOM
SCBOM_chi = pd.read_excel("/home/yifei/Documents/BOM_tool/SCBOM.xlsx")
print("Supply Chain BOM Shape: ",SCBOM_chi.shape)
SCBOM_chi.head(20)


Supply Chain BOM Shape:  (6253, 47)


,Key,Identifier,PartNumber,PartRevision,LegacyPartNumber,LegacyPartRevision,System,SubSystem,Description,QTY,...,LastPickupDate,SC_PackagingeProposalApproved,PartStatus,PartActive,Maturity,RevisionComment,Configuration,EngineeringReleaseEstimate,EngineeringReleaseActual,AssemblyStation
0,AA-YD004238NN-00A.1EC001_04_999_0059VP,NaN,AA-YD004238NN-00,A.1,EC001_04_999_0059,NaN,EXTERIOR -STANDARD PARTS,REAR BUMPER,FLANGE NUT M6,6.0,...,NaT,False,Removed,Inactive,NaN,Set2 Count = 1 Set3 Count = 1,NaN,NaN,NaN,NaN
1,(2)7052134730A.1VP,11|3|8|1|8,(2)7052134730,A.1,NaN,NaN,NaN,NaN,NaN,6.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
2,(2)7282-6443-40_Dup4A.1VP,11|3|8|1|18,(2)7282-6443-40_Dup4,A.1,NaN,NaN,NaN,NaN,NaN,3.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
3,(2)7283-3441-40_prd-BYT-00606114A.1VP,11|3|17|2|5,(2)7283-3441-40_prd-BYT-00606114,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
4,_DMU7282-6453-60A.1VP,11|3|17|2|10,_DMU7282-6453-60,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
5,_DMU7283645440.1A.1VP,11|3|17|2|7,_DMU7283645440.1,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
6,_FLT2471952XA.1VP,4|4|7|1,_FLT2471952X,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
7,_FLT2471954XA.1VP,4|4|8|1,_FLT2471954X,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
8,_FLT2471955XA.1VP,4|4|9|1,_FLT2471955X,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
9,_FLT2492062XA.1VP,4|4|3|1,_FLT2492062X,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN


In [6]:
SCBOM_chi.columns

Index(['Key', 'Identifier', 'PartNumber', 'PartRevision', 'LegacyPartNumber',
       'LegacyPartRevision', 'System', 'SubSystem', 'Description', 'QTY',
       'Type', 'Engineer', 'Program', 'PurchasePartType', 'NominatedSupplier',
       'ConfirmedSupplier', 'GSM', 'RFQSent', 'RFQReceived', 'LOIReceived',
       'SourcingCouncilScheduledDate', 'SourcingCouncilAprovedDate', 'DSA',
       'GTC', 'AwardLetter', 'PurchaseRequistion', 'PurchaseOrder',
       'ManufacturingLT', 'EngineeringReleaseNeededBy', 'CreatedDate',
       'GSMComment', 'ModifiedDate', 'FirstPickupDate', 'CCCRequired',
       'PieceCost', 'CustomComplianceCompleted', 'SubCommodity',
       'LastPickupDate', 'SC_PackagingeProposalApproved', 'PartStatus',
       'PartActive', 'Maturity', 'RevisionComment', 'Configuration',
       'EngineeringReleaseEstimate', 'EngineeringReleaseActual',
       'AssemblyStation'],
      dtype='object')

In [7]:
PreAP_BOM_chi = SCBOM_chi[SCBOM_chi["Program"]=="PAP"]
print("Pre AP BOM shape: ", PreAP_BOM_chi.shape)
AP_BOM_chi = SCBOM_chi[SCBOM_chi["Program"]=="AP"]
print("AP BOM shape: ", AP_BOM_chi.shape)
VP_BOM_chi = SCBOM_chi[SCBOM_chi["Program"]=="VP"]
print("VP BOM shape: ", VP_BOM_chi.shape)

#Active VP BOM with latest part revision
VP_BOM_Activate_chi = VP_BOM_chi[(VP_BOM_chi["PartActive"]=="Active")]
print("VP Active BOM: ", VP_BOM_Activate_chi.shape)

VPM_BOM_Activate_chi_removePhantomPart = VP_BOM_Activate_chi[VP_BOM_Activate_chi["Type"]!="Phantom"]
print("removed phatom", VPM_BOM_Activate_chi_removePhantomPart.shape)

                                                   


Pre AP BOM shape:  (863, 47)
AP BOM shape:  (0, 47)
VP BOM shape:  (5390, 47)
VP Active BOM:  (5064, 47)
removed phatom (5000, 47)


In [8]:
#Load shaolong's SC BOM which has the most sourcing information updated by buyers and SQEs. 
SCBOM_shaolong = pd.read_excel("/home/yifei/Documents/BOM_tool/Shaolong BOM.xlsx", index_col=None)
# SCBOM_shaolong.reset_index()
SCBOM_shaolong

,No.,SYSTEM,Commodity Package,Count Commodity,Description,BYTON P/N,ENGINEER,NJ IPM,GSM,Senior GSM,...,Hard tool Completion,Home line,FTN,Grain,AAR,Run at rate,PV Completion,Phase 1 PSW,Phase 3 PSW,Comments
0,1900-01-01,LOW VOLTAGE SYSTEMS (E/E),12 V OUTLET,1.0,12V POWER OUTLET,AA-LE000742NN-00,noopur.divekar@byton.com,NaN,Jennie.Lu,Veronica.Park,...,2018-11-05 00:00:00,2018-11-12 00:00:00,,,,2019-03-11 00:00:00,2019-03-25 00:00:00,2019-06-21 00:00:00,,
1,1900-01-02,LOW VOLTAGE SYSTEMS (E/E),12 V OUTLET,0.0,CIGAR LIGHTER SOCKET,AA-LE001138NN-00,noopur.divekar@byton.com,NaN,Jennie.Lu,Veronica.Park,...,2018-11-05 00:00:00,2018-11-12 00:00:00,,,,2019-03-11 00:00:00,2019-03-25 00:00:00,2019-06-21 00:00:00,,
2,1900-01-03,LOW VOLTAGE SYSTEMS (E/E),12 V OUTLET,0.0,CIGAR LIGHTER POPOUT,AA-LE001137NN-00,noopur.divekar@byton.com,NaN,Jennie.Lu,Veronica.Park,...,2018-11-05 00:00:00,2018-11-12 00:00:00,,,,2019-03-11 00:00:00,2019-03-25 00:00:00,2019-06-21 00:00:00,,
3,1900-01-04,LOW VOLTAGE SYSTEMS (E/E),12V BATTERY,1.0,12V BATTERY,AA-LA000707NN-00,rockymark.juan@byton.com,NaN,Jennie.Lu,Veronica.Park,...,,,,,,,,,,"supplier should be ""C&D"""
4,1900-01-05,LOW VOLTAGE SYSTEMS (E/E),230 V outlet,1.0,230V HIGH VOLTAGE POWER OUTLET,AA-LE000746NN-00,noopur.divekar@byton.com,NaN,Jennie.Lu,Veronica.Park,...,2018-11-05 00:00:00,2018-11-12 00:00:00,,,,2019-03-11 00:00:00,2019-03-25 00:00:00,2019-06-21 00:00:00,,
5,1900-01-06,INTELLIGENT CAR ENTERTAINMENT (ICE),4 ARRAY MEMS MICROPHONE,1.0,4 ARRAY MEMS MICROPHONE,AA-ND000703FN-00,CHET.DIKEN@BYTON,Ryan.Dong,Ryan.Dong,Veronica.Park,...,,,,,,,,,,"Byton Inhouse design, production supplier not ..."
6,1900-01-07,CHASSIS,ACCELERATOR PEDAL,1.0,ACCELERATOR PEDAL LHD ASSY,AA-EB110100FN-00,gary.waite@ricardo.com,NaN,Tom.Peleg,Tom.Peleg,...,,,,,,,,,,Parts already SOP for Other customer
7,1900-01-08,INTELLIGENT CAR ENTERTAINMENT (ICE),ACOUSTIC VEHICLE ALERT SYSTEM,1.0,ACOUSTIC VEHICLE ALERT SYSTEM,AA-NH003864FN-03,MARKUS.KOCH@BYTON.COM,NaN,Veronica.Park,Veronica.Park,...,,,,,,,,,,Not sourced for SOP
8,1900-01-09,INTERIOR,ACU,1.0,AIRBAG CONTROL UNIT,AA-DJ000855NN-00,JOHN.SCHEICK@BYTON.COM,Kun Fang,Kun Fang,Shaolong.Huang,...,,,,,,,,,,No feedback form supplier
9,1900-01-10,EXTERIOR,AERO PNL D-PILLAR,1.0,ASSY AERO PNL D-PILLAR LH,AA-CF003398NL-00,Mark Bruderick,Mark Shen,Mark Shen,Harvey.Li,...,,,,,,,,,,


In [9]:
SCBOM_shaolong.columns.values

array(['No.', 'SYSTEM', 'Commodity Package', 'Count Commodity',
       'Description', 'BYTON P/N', 'ENGINEER', 'NJ IPM', 'GSM',
       'Senior GSM', 'VP\nSUPPLIER\n(Name)', 'Supplier Sourced?',
       'Supplier \nContact Person', 'Supplier \nContact E-mail',
       'Supplier Location', 'Over Sea Plant?',
       'FOB / FAC &Where?\n(if over sea)', 'Tier', 'Ship to',
       'GTC & Contract', 'SQE Priority?', 'Risky Esclation', 'JIS?',
       'Oversea Pick Up Location', 'Package cost confirmed?',
       'Transport Price quoted?', 'Daily Production Capability',
       'Order Leader Time', 'Mnimum Order Quantity', 'SQE', 'Priority',
       'Reason for Priority', 'APQP \nKICK OFF', 'Unnamed: 33',
       'Tooling\nLead time', 'Tool Kick off Plan', 'Hard tool Completion',
       'Home line', 'FTN', 'Grain', 'AAR', 'Run at rate', 'PV Completion',
       'Phase 1 PSW', 'Phase 3 PSW', 'Comments'], dtype=object)

In [10]:
print("Supply Chain BOM Shape: ",SCBOM_shaolong.shape)

Supply Chain BOM Shape:  (1516, 46)


## Step 0: Create a blank Supply Chain BOM using EBOM

In [11]:
# copy EBOM to the new SCBOM
print("EBOM shape is: ", EBOM.shape)
SCBOM = EBOM.copy()
print("SCBOM shape is: ", SCBOM.shape)
#specify SCBOM columns
SCBOM.columns.values

EBOM shape is:  (3164, 14)
SCBOM shape is:  (3164, 14)


array(['Identifier', 'Title', 'Revision', 'Description', 'QTY', 'UOM',
       'Purchased Part Type', 'Maturity', 'Part Type', 'System',
       'SubSystem', 'Legacy Part Number', 'Legacy Part Revision',
       'Configuration'], dtype=object)

In [12]:
ebom_column_header = EBOM.columns.values

sourcing_column_header = ["Commodity Package", "Engineer", "GSM", "Senior GSM", "RFQ Sent", "RFQ Received", 
                          "Sourcing Council Scheduled Date", "Sourcing Council Aproved Date", 
                          "Supplier Name", "Sourcing Status", "Supplier Category", "Supplier Contact Person",
                          "Supplier Contact Email", "Supplier Manufacturing Region", 
                          "Supplier Manufacturing Country", "Incoterm", "GTC", "LOI", "Purchase Contract", 
                          "JIS", "Tier", "Ship to", "Daily Production Capability","Order Leader Time ","MOQ",
                          "Purchae Requisition","Purchase Order", "First Pickup Date", "Last Pickup Date", 
                         ]

cost_column_header = ["Unit Price", "Tooling Cost", "ED&T","Oversea Pick Up Location","Package cost confirmed?",
                      "Transport Price quoted?","Total Investment","Currency",    
                     ]

logistics_column_header = ["HS code Export", "HS code Import", "Regulatory Requirements (Y/N)", "Logistic Planned (Y/N)", 
                          "Packaging Proposal Approved","EDI Setup"
                          ]

quality_column_header = ["SQE", "Priority", "APQP Kick Off", "APQP Status","Tooling Lead Time", 
                     "Tooling Kick Off Date", "Tooling Completion Date", "Home Line", "FTN", 
                     "Grain", "AAR", "Run At Rate", "PV Completion Date", "Phase 1 PSW Date", 
                     "Phase 3 PSW Date", "Comments"
                        ]

log_column_header = ["Engineering Release Estimtae", "Engineering Release Actual", "Assembly Station", 
                     "Maturity", "Configuration", "Part Creation Date", "Last Modified Date", "Part Status", 
                     "Part Active"
                    ]

column_header = sourcing_column_header + cost_column_header + logistics_column_header + quality_column_header + log_column_header

In [13]:
# add additional columns
def add_columns(df, columns):
    df2 = pd.DataFrame(columns=columns)
    df = df.append(df2, sort=False, verify_integrity=True)
    return df

In [14]:
SCBOM = add_columns(SCBOM, column_header)
SCBOM.shape

(3164, 80)

## Step 1: Update new SCBOM using Shaolong's BOM

In [15]:
df = pd.DataFrame({'BoolCol': [True, False, False, True, True], 'Name': [1,2,3,4,5]},
       index=[10,20,30,40,50])
idx = df.index[df['BoolCol']]
idx

Int64Index([10, 40, 50], dtype='int64')

In [16]:
df.loc[idx[0]]["Name"]

1

In [17]:
df

,BoolCol,Name
10,True,1
20,False,2
30,False,3
40,True,4
50,True,5


In [18]:
df.loc[(df["Name"]==2) & (df["BoolCol"]==False)].index.values

array([20])

In [19]:
df.loc[[10], ["Name"]] = df.loc[[20], 'Name'].values
df

,BoolCol,Name
10,True,2
20,False,2
30,False,3
40,True,4
50,True,5


In [20]:
# save dataframe to Excel
def save(df):
    #https://stackoverflow.com/questions/28837057/pandas-writing-an-excel-file-containing-unicode-illegalcharactererror
    df = df.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)

    writer = pd.ExcelWriter('Supply Chain BOM.xlsx')
    df.to_excel(writer, sheet_name="Supply Chain BOM", na_rep="" )
    writer.save()

#     df.to_csv("Supply Chain BOM.csv",quoting=csv.QUOTE_NONE, escapechar="\\")

In [21]:
# search PN and Rev in shaolong's SCBOM
def search(PN):
    # shaolong's data doesn't have Rev.
#     df = SCBOM_shaolong.loc[(SCBOM_shaolong["BYTON P/N"]==PN) & (SCBOM_shaolong["BYTON REVISION"]==Rev)]
    df = SCBOM_shaolong.loc[(SCBOM_shaolong["BYTON P/N"]==PN)]
    index = df.index
    if (len(index)==1):
        return index
    else:
        #return the first index if duplicate PN found
        return index[0:1]
        

In [22]:
print(search("AA-AF003416NN-00"))

Int64Index([261], dtype='int64')


In [23]:
print(search("AA-YF000148NN-00"))

Int64Index([1105], dtype='int64')


In [24]:
def replace_bracket(data):
    value = str(data).replace('[', '').replace(']', '').replace('\'', '')
    if value == 'nan':
        return ""
    return value

def change_datetime_to_string(data):
    if data.size == 0:
        return ""
    elif (data == np.array([' '])):
        return ""
    elif data == "nan":
        return ""
    else:
        return data[0]
    
#     if data == np.array([' ']):
#         return ""
#     if data == "nan":
#         return ""
#     else:
#         return data[0].strftime('%m/%d/%Y')

In [25]:
# copy Shaolong's BOM info and paste into SCBOM
def copy_and_paste_row(df1, index1, df2, index2):
    #sourcing
    df1.loc[index1,["Engineer"]] = replace_bracket(df2.loc[index2,["ENGINEER"]].values)
    df1.loc[index1,["GSM"]] = replace_bracket(df2.loc[index2,["GSM"]].values)
    df1.loc[index1,["Senior GSM"]] = replace_bracket(df2.loc[index2,["Senior GSM"]].values)
    df1.loc[index1,["Supplier Name"]] = replace_bracket(df2.loc[index2,["VP\nSUPPLIER\n(Name)"]].values)
    df1.loc[index1,["Sourcing Status"]] = replace_bracket(df2.loc[index2,["Supplier Sourced?"]].values)
    df1.loc[index1,["Supplier Contact Person"]] = replace_bracket(df2.loc[index2,["Supplier \nContact Person"]].values)
    df1.loc[index1,["Supplier Contact Email"]] = replace_bracket(df2.loc[index2,["Supplier \nContact E-mail"]].values)
    df1.loc[index1,["Supplier Manufacturing Country"]] = replace_bracket(df2.loc[index2,["Supplier Location"]].values)
    df1.loc[index1,["GTC"]] = replace_bracket(df2.loc[index2,["GTC & Contract"]].values)

    
    #Logistics
    
    #SQE
    df1.loc[index1,["SQE"]] = replace_bracket(df2.loc[index2,["SQE"]].values)
    df1.loc[index1,["Priority"]] = replace_bracket(df2.loc[index2,["SQE Priority?"]].values)
    df1.loc[index1,["APQP Kick Off"]] = replace_bracket(df2.loc[index2,["APQP \nKICK OFF"]].values)
    df1.loc[index1,["APQP Status"]] = replace_bracket(df2.loc[index2,["Unnamed: 33"]].values)
    df1.loc[index1,["Tooling Lead Time"]] = replace_bracket(df2.loc[index2,["Tooling\nLead time"]].values)
    df1.loc[index1,["Tooling Kick Off Date"]] = change_datetime_to_string(df2.loc[index2,["Tool Kick off Plan"]].values)
    df1.loc[index1,["Tooling Completion Date"]] = change_datetime_to_string(df2.loc[index2,["Hard tool Completion"]].values)
    df1.loc[index1,["Home Line"]] = change_datetime_to_string(df2.loc[index2,["Home line"]].values)
    df1.loc[index1,["FTN"]] = replace_bracket(df2.loc[index2,["FTN"]].values)
    df1.loc[index1,["Grain"]] = replace_bracket(df2.loc[index2,["Grain"]].values)
    df1.loc[index1,["FTN"]] = replace_bracket(df2.loc[index2,["FTN"]].values)
    df1.loc[index1,["AAR"]] = replace_bracket(df2.loc[index2,["AAR"]].values)
    df1.loc[index1,["Run At Rate"]] = change_datetime_to_string(df2.loc[index2,["Run at rate"]].values)
    df1.loc[index1,["PV Completion Date"]] = change_datetime_to_string(df2.loc[index2,["PV Completion"]].values)
    df1.loc[index1,["Phase 1 PSW Date"]] = change_datetime_to_string(df2.loc[index2,["Phase 1 PSW"]].values)
    df1.loc[index1,["Phase 3 PSW Date"]] = change_datetime_to_string(df2.loc[index2,["Phase 3 PSW"]].values)
    df1.loc[index1,["Comments"]] = replace_bracket(df2.loc[index2,["Comments"]].values)
    
    
    #log
#   log_column_header = ["Engineering Release Estimtae", "Engineering Release Actual", "Assembly Station", 
#                      "Maturity", "Configuration", "Part Creation Date", "Last Modified Date", "Part Status", 
#                      "Part Active"]
    df1.loc[index1,["Part Status"]] = "Latest revision"
    df1.loc[index1,["Part Active"]] = "Active"
        
        
    return df1
    
    
    
     

In [26]:
print(str(SCBOM_shaolong.loc[1000, ["GSM"]].values))

['Joseph Zhu']


In [27]:
print(str(SCBOM_shaolong.loc[1000, ["GSM"]].values).replace('[', '').replace(']', '').replace('\'', ''))

Joseph Zhu


In [28]:
SCBOM_shaolong.loc[:, ["Tool Kick off Plan"]].values

array([[datetime.datetime(2018, 9, 13, 0, 0)],
       [datetime.datetime(2018, 9, 13, 0, 0)],
       [datetime.datetime(2018, 9, 13, 0, 0)],
       ...,
       [' '],
       [' '],
       [' ']], dtype=object)

In [29]:
SCBOM_shaolong.loc[41, ["Tool Kick off Plan"]].values[0]

datetime.datetime(2018, 9, 15, 0, 0)

In [30]:
# update SCBOM with inputs from Shaolong's BOM
for index, row in SCBOM.iterrows():
    #loop through EBOM by PN
    PN = row["Title"]
    Rev = row["Revision"]
    #find out if this PN and Rev exist in Shaolong's BOM, and return the index
    index_shaolong = search(PN)
    #if not found
    if (index_shaolong.values.size == 0):
        #do nothing
        continue
    #if PN found, update SCBOM with Shaolong's BOM info
    else:
        #copy shaolong's BOM info to SCBOM
        copy_and_paste_row(SCBOM, index, SCBOM_shaolong, index_shaolong)


        
            
        
        
    
    

In [31]:
SCBOM.loc[[21]]

,Identifier,Title,Revision,Description,QTY,UOM,Purchased Part Type,Maturity,Part Type,System,...,Phase 1 PSW Date,Phase 3 PSW Date,Comments,Engineering Release Estimtae,Engineering Release Actual,Assembly Station,Part Creation Date,Last Modified Date,Part Status,Part Active
21,1|2|1|1|1|4|1,AA-YF000148NN-00,A.1,HEXAGONAL RIVET NUT M6,1.0,EA EACH,Buy,Released,Part,Y Fasteners,...,,,,NaN,NaN,NaN,NaN,NaN,Latest revision,Active


In [32]:
save(SCBOM)

## Create a new jupyter notebook for this step
### Step 2: Maintain and update SCBOM using updated EBOM

We need to maintain the integrity of the EBOM, so everytime EBOM is updated, a new corresponding supply chain BOM is created with identifical information copied from EBOM. Then the supply chain BOM is updated with information from previous supply chain BOM, parts info whose PN and Rev are identical with EBOM will be copy and pasted in, parts that are no longer identifical will be marked as inactive and/or removed parts. 

In [1]:
import pandas as pd
import numpy as np
import xlrd, openpyxl
import csv, datetime

In [2]:
EBOM_PATH = './Data/AA-ZZ000001NN-03_VP BoM_9-21-18 at 3.42 PM.xlsx'
SCBOM_PATH = './Data/Supply Chain BOM V2 (version 1).xlsx'

In [3]:
#load EBOM
def load():
    # load Engineering BOM
    EBOM = pd.read_excel(EBOM_PATH, sheet_name="BOM", skiprows=[0])
    EBOM.columns = EBOM.columns.str.replace("\(R\)\ ", "") # trim (R) away from the header
    EBOM = EBOM.reset_index(drop=True)
    # load Supply Chain BOM
    SCBOM = pd.read_excel(SCBOM_PATH, sheet_name="Supply Chain BOM")
    SCBOM = SCBOM.reset_index(drop=True)
    return EBOM, SCBOM

EBOM, SCBOM = load()

In [4]:
SCBOM_columns = SCBOM.columns.tolist()
SCBOM_columns_size = len(SCBOM_columns)
EBOM_columns = EBOM.columns.tolist()
EBOM_columns_size = len(EBOM_columns)

In [5]:
print("EBOM shape: ", EBOM.shape)
print("SCBOM shape: ", SCBOM.shape)
print("# of columns in EBOM: ", EBOM_columns_size)
print("# of columns in SCBOM: ", SCBOM_columns_size)

EBOM shape:  (4570, 15)
SCBOM shape:  (3164, 78)
# of columns in EBOM:  15
# of columns in SCBOM:  78


In [6]:
# create a updated SCBOM with columns from SCBOM and data from EBOM
SCBOM_updated = EBOM.copy()
for each in SCBOM.columns.tolist()[15:]:
    SCBOM_updated[each] = ""
SCBOM_updated.shape

(4570, 78)

In [7]:
# search PN and Rev in SCBOM
def search(df, PN, Rev):
    df = df.loc[(df["Title"]==PN) & (df["Revision"]==Rev)]
    index = df.index.values
    
    # only one entry found
    if (len(index)==1):
        return index   
    
    # multiple entries found
    elif (len(index)>1):
        #return the first index if duplicate PN and Rev found
        return index.tolist()   # returned indexes are a numpy.ndarray, we convert to a list and get the first value
    
    # no entry found
    else:
        return None
        

In [8]:
print(search(EBOM, 'AA-MB001705NN-00', 'D.1'))
print(len(search(EBOM, 'AA-MB001705NN-00', 'D.1')))

[4502]
1


In [9]:
print(search(EBOM, 'AA-MB001703NN-00', 'B.1'))
print(len(search(EBOM, 'AA-MB001703NN-00', 'B.1'))

SyntaxError: unexpected EOF while parsing (<ipython-input-9-34332973a15c>, line 2)

In [10]:
# def replace_bracket(data):
#     value = str(data).replace('[', '').replace(']', '').replace('\'', '')
#     if value == 'nan':
#         return ""
#     return value

# def change_datetime_to_string(data):
#     if data.size == 0:
#         return ""
#     elif (data == np.array([' '])):
#         return ""
#     elif data == "nan":
#         return ""
#     else:
#         return data[0]
    
# #     if data == np.array([' ']):
# #         return ""
# #     if data == "nan":
# #         return ""
# #     else:
# #         return data[0].strftime('%m/%d/%Y')

In [11]:
# # copy EBOM info and paste into SCBOM
# def copy_and_paste_row(df1, index1, df2, index2):
#     # copy df2 info into df1
#     df1.loc[index1,["Identifier"]] = replace_bracket(df2.loc[index2,["Identifier"]].values)
#     df1.loc[index1,["Title"]] = replace_bracket(df2.loc[index2,["Title"]].values)
#     df1.loc[index1,["Revision"]] = replace_bracket(df2.loc[index2,["Revision"]].values)
#     df1.loc[index1,["Description"]] = replace_bracket(df2.loc[index2,["Description"]].values)
#     df1.loc[index1,["QTY"]] = replace_bracket(df2.loc[index2,["QTY"]].values)
#     df1.loc[index1,["UOM"]] = replace_bracket(df2.loc[index2,["UOM"]].values)
#     df1.loc[index1,["Purchased Part Type"]] = replace_bracket(df2.loc[index2,["Purchased Part Type"]].values)
#     df1.loc[index1,["UOM"]] = replace_bracket(df2.loc[index2,["UOM"]].values)
#     df1.loc[index1,["Maturity"]] = replace_bracket(df2.loc[index2,["Maturity"]].values)
#     df1.loc[index1,["Part Type"]] = replace_bracket(df2.loc[index2,["Part Type"]].values)
#     df1.loc[index1,["System"]] = replace_bracket(df2.loc[index2,["System"]].values)
#     df1.loc[index1,["SubSystem"]] = replace_bracket(df2.loc[index2,["SubSystem"]].values)
#     df1.loc[index1,["Legacy Part Number"]] = replace_bracket(df2.loc[index2,["Legacy Part Number"]].values)
#     df1.loc[index1,["Legacy Part Revision"]] = replace_bracket(df2.loc[index2,["Legacy Part Revision"]].values)
#     df1.loc[index1,["Configuration"]] = replace_bracket(df2.loc[index2,["Configuration"]].values)

#     return df1

In [12]:
# copy EBOM info and paste into SCBOM
def copy_and_paste_row(df1, index1, df2, index2):  # index1 and index2 are int
    # copy df2 info into df1
    offset = df2.shape[1]
    for i in range(offset-1, len(df2.columns.tolist()), 1):
        df1.loc[index1, df1.columns.tolist()[i]] = df2.loc[index2, df2.columns.tolist()[i]]
        
    return df1

In [285]:
# save dataframe to Excel
def save(df):
    #https://stackoverflow.com/questions/28837057/pandas-writing-an-excel-file-containing-unicode-illegalcharactererror
    df = df.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)

    writer = pd.ExcelWriter('Updated Supply Chain BOM.xlsx')
    df.to_excel(writer, sheet_name="Updated Supply Chain BOM", na_rep="" )
    writer.save()

#     df.to_csv("Supply Chain BOM.csv",quoting=csv.QUOTE_NONE, escapechar="\\")

In [289]:
# load EBOM and SCBOM
EBOM, SCBOM = load()

# create a updated SCBOM with columns from SCBOM and data from EBOM
SCBOM_updated = EBOM.copy()
for each in SCBOM.columns.tolist()[14:]:
    SCBOM_updated[each] = ""
SCBOM_updated.shape



(4570, 78)

In [365]:
EBOM.columns

Index(['Identifier', 'Title', 'Revision', 'Description', 'QTY', 'UOM',
       'Purchased Part Type', 'Maturity', 'Part Type', 'System', 'SubSystem',
       'Type', 'Legacy Part Number', 'Legacy Part Revision', 'Configuration'],
      dtype='object')

In [368]:
SCBOM.columns.tolist()[14:]

['Commodity Package',
 'Engineer',
 'GSM',
 'Senior GSM',
 'Supplier Name',
 'Sourcing Status',
 'Active/Inactive',
 'Supplier Contact Person',
 'Supplier Contact Email',
 'Supplier Manufacturing Country',
 'Supplier Manufacturing City',
 'Incoterm',
 'GTC(Y/N)',
 'LOI(Y/N)',
 'Purchase Contract(Y/N)',
 'JIS(Y/N)',
 'Tier',
 'Ship to',
 'Daily Production Capability',
 'Order Lead Time ',
 'MOQ',
 'Purchae Requisition',
 'Purchase Order',
 'First Pickup Date',
 'Last Pickup Date',
 'Unit Price',
 'Tooling Cost',
 'ED&T',
 'Oversea Pick Up Location',
 'Package Cost Estimate',
 'Transport Price quoted?',
 'Total Investment',
 'Origin Currency',
 'HS code Export',
 'HS code Import',
 'Regulatory Requirements (Y/N)',
 'Logistic Planned (Y/N)',
 'Packaging Proposal Approved(Y/N)',
 'EDI Setup',
 'Duties',
 'Taxes',
 'SQE',
 'Priority(H/M/L)',
 'APQP Kick Off Date',
 'APQP Status',
 'Tooling Lead Time',
 'Tooling Kick Off Date',
 'Tooling Completion Date',
 'Home Line',
 'FTN',
 'Grain',
 'AA

In [299]:
# loop through SCBOM 
for index, row in SCBOM.iterrows():
    PN = row["Title"]
    Rev = row["Revision"]
#     Identifier = row["Identifier"] # same part could be structured differently but the part is the same, so if same part found at multiple places, we will just pick one

    # search if this part exists in SCBOM_updated, return the index in SCBOM_updated
    index_SCBOM_updated = search(SCBOM_updated, PN, Rev)
    print(index, index_SCBOM_updated)

0 [1]
1 [2]
2 None
3 [5]
4 [6]
5 [7]
6 [11]
7 [12]
8 [13]
9 [14]
10 [17]
11 [18]
12 [21]
13 [23]
14 [25]
15 [26]
16 [27]
17 [28]
18 [30]
19 [31]
20 [35]
21 [37]
22 [40]
23 [41]
24 [42, 60]
25 [43]
26 [50]
27 [44]
28 [45]
29 [46]
30 [51]
31 [52]
32 [54]
33 [55]
34 [58]
35 [59]
36 [61]
37 None
38 [63]
39 [66]
40 [71]
41 [72]
42 [73]
43 [74]
44 [68]
45 [69]
46 [70]
47 [65]
48 [76]
49 [78]
50 [79]
51 [80]
52 [82]
53 [84]
54 [85]
55 None
56 None
57 [88]
58 [89]
59 [91]
60 [93]
61 [97]
62 [99]
63 [100]
64 [101]
65 [103]
66 [104]
67 [331]
68 [332]
69 [106]
70 [107]
71 [110]
72 [112]
73 [113]
74 [116]
75 [118]
76 [119]
77 [120]
78 [121]
79 [96, 123, 204, 226]
80 [126]
81 [128]
82 [130]
83 [131]
84 [133]
85 None
86 [134]
87 [135, 153, 238, 257]
88 [138]
89 [140]
90 [141]
91 [145]
92 [147]
93 [159]
94 [162]
95 [148]
96 [149]
97 [150]
98 [151]
99 [152]
100 [95]
101 [165]
102 [170]
103 [177]
104 [178]
105 [172]
106 [173]
107 [174]
108 [168, 266]
109 [169, 267]
110 [167]
111 [181]
112 [185]
113 [18

1473 None
1474 None
1475 None
1476 None
1477 None
1478 [2450]
1479 [2451]
1480 [2452]
1481 [2453]
1482 [2454]
1483 [2455]
1484 [2456]
1485 [2457]
1486 [2462]
1487 [2467]
1488 [2470]
1489 [2473]
1490 [2474]
1491 [2480]
1492 [2479]
1493 [2475]
1494 [2476]
1495 [2484]
1496 [2485]
1497 [2486]
1498 [2487]
1499 [2488]
1500 [2489]
1501 [2605]
1502 [2606]
1503 [2607]
1504 None
1505 [2610]
1506 [2611]
1507 [2614]
1508 [2615]
1509 [2631]
1510 None
1511 [2647]
1512 [2652]
1513 [2656]
1514 [2654]
1515 [2621]
1516 [2619]
1517 [2617]
1518 [2650, 2660]
1519 [2623]
1520 [2633]
1521 [2634]
1522 [2635]
1523 [2636]
1524 [2629]
1525 [2627]
1526 [2642]
1527 [2643, 2645]
1528 [2648]
1529 [2661]
1530 None
1531 [2657]
1532 [2701]
1533 [2703]
1534 None
1535 None
1536 None
1537 None
1538 None
1539 None
1540 None
1541 None
1542 None
1543 None
1544 None
1545 None
1546 None
1547 None
1548 None
1549 None
1550 None
1551 None
1552 None
1553 None
1554 None
1555 None
1556 None
1557 [2784, 2834, 2842, 2922, 2982]
1558 N

2225 [4139, 4298]
2226 [4140]
2227 [4141, 4222]
2228 [4142]
2229 [4143, 4223]
2230 [4144, 4224]
2231 [4145, 4225]
2232 None
2233 [4146, 4226]
2234 [4147, 4227]
2235 [4148, 4228]
2236 [4149, 4229]
2237 [4150, 4230]
2238 [4151, 4231]
2239 [4152, 4232]
2240 [4153, 4233]
2241 [4154, 4234]
2242 [4155, 4235]
2243 [4158, 4238]
2244 [4159, 4239]
2245 [4160, 4240]
2246 [4161, 4241]
2247 [4163, 4243]
2248 [4164]
2249 [4167, 4244]
2250 [4174, 4251]
2251 [4176, 4256]
2252 [4177, 4255]
2253 [4257]
2254 [4258]
2255 [4259]
2256 [4292]
2257 [4180, 4262]
2258 [4190, 4303]
2259 [4191, 4273]
2260 [4192, 4275]
2261 [4193, 4276]
2262 [4194, 4277]
2263 [4195, 4278]
2264 [4196, 4279]
2265 [4197, 4300]
2266 [4198, 4301]
2267 [4199, 4302]
2268 [4200, 4304]
2269 None
2270 [4202, 4306]
2271 [4206]
2272 [4210]
2273 [4253]
2274 [4254]
2275 [4272]
2276 [4274]
2277 [4280]
2278 [4281]
2279 [4282]
2280 [4283]
2281 [4284]
2282 [4285]
2283 [4286]
2284 [4287]
2285 [4288]
2286 [4289]
2287 [4290]
2288 [4291]
2289 [4295]
22

2716 [1385]
2717 [1576]
2718 [1584]
2719 [1623]
2720 [1661]
2721 [1696]
2722 None
2723 [1773]
2724 [1787]
2725 [1806, 1809]
2726 [1870]
2727 None
2728 [1927, 1943, 1958, 1971]
2729 None
2730 None
2731 None
2732 None
2733 None
2734 None
2735 None
2736 None
2737 None
2738 None
2739 None
2740 None
2741 None
2742 None
2743 None
2744 None
2745 None
2746 None
2747 [3021]
2748 None
2749 None
2750 None
2751 None
2752 None
2753 None
2754 None
2755 None
2756 None
2757 None
2758 None
2759 None
2760 None
2761 None
2762 None
2763 None
2764 None
2765 None
2766 None
2767 None
2768 None
2769 None
2770 None
2771 None
2772 None
2773 None
2774 None
2775 None
2776 None
2777 None
2778 None
2779 None
2780 None
2781 None
2782 None
2783 None
2784 None
2785 None
2786 None
2787 None
2788 None
2789 None
2790 None
2791 None
2792 None
2793 None
2794 None
2795 None
2796 None
2797 None
2798 None
2799 None
2800 None
2801 None
2802 None
2803 None
2804 None
2805 None
2806 None
2807 None
2808 None
2809 None
2810 None
28

In [361]:
a = pd.DataFrame({"A": [1,2,3], "B": [4,5,6]})
b = pd.DataFrame({"A": [7,8,9], "B": [0,1,2]})
# a.append(b.loc[0], ignore_index=True)
b.loc[0, "B"] = 1
b.loc[0]
a.append(b.loc[0], ignore_index=True)



,A,B
0,1,4
1,2,5
2,3,6
3,7,1


In [358]:
# loop through SCBOM 
for index, row in SCBOM.iterrows():
    PN = row["Title"]
    Rev = row["Revision"]
    # same part could be structured differently but the part is the same, 
    # so if same part found at multiple places, we will just pick one
#     Identifier = row["Identifier"] 

    # search if this part exists in SCBOM_updated, return the index in SCBOM_updated
    # search() returned a list or None type
    index_SCBOM_updated = search(SCBOM_updated, PN, Rev)

    #not found, deactivate the part, then append this part to SCBOM_updated
    if (index_SCBOM_updated == None):
        SCBOM.loc[index, ["Part Active"]] = "Inactivate"
        SCBOM.loc[index, ["Part Status"]] = "Removed"
        SCBOM.loc[index, ["Last Modified Date"]] = datetime.datetime.now()
        SCBOM_updated = SCBOM_updated.append(SCBOM.loc[index], ignore_index=True)


    # found one entry, copy the information to updated SCBOM    
    elif (len(index_SCBOM_updated) == 1):
        index_SCBOM_updated = index_SCBOM_updated[0]
        SCBOM_updated = copy_and_paste_row(SCBOM_updated, index_SCBOM_updated, SCBOM, index)
#         SCBOM_updated.loc[index_SCBOM_updated, ["Part Creation Date"]] = "9/21/2018"

    # found multiple entries, copy the information to each of the entries in SCBOM_updated
    else:
        for each in index_SCBOM_updated:
            SCBOM_updated = copy_and_paste_row(SCBOM_updated, each, SCBOM, index)


save(SCBOM_updated)

print("\nAfter Sync")
print("EBOM shape: ", EBOM.shape)
print("SCBOM shape: ", SCBOM.shape)
print("SCBOM_updated ", SCBOM_updated.shape)

execution_time = time.time() - start_time
print("This script took \n--- {} seconds ---".format(execution_time)

1
1
AA-AZ000002NN-03
AA-AZ000002NN-03
A.1
A.1
PHTM BIW VP
PHTM BIW VP
1
1
EA EACH
EA EACH
nan
nan
In Work
In Work
Phantom
Phantom
A BIW
A BIW
A Z NA
A Z NA
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
N
N
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
NaT
NaT
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
NaT
NaT
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
1|1
1|1
AA-AF000002NN-03
AA-AF000002NN-03
A.1
A.1
PHTM BIW ROOF STRUCTURE VP
PHTM BIW ROOF STRUCTURE VP
1
1
EA EACH
EA EACH
nan
nan
In Work
In Work
Phantom
Phantom
A BIW
A BIW
A F Roof Structure
A F Roof Structure
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
N
N
nan
nan
nan
nan
nan
nan
nan
na

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
NaT
NaT
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
NaT
NaT
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
1|2|1|1|1
1|2|1|1|1
AA-AE003418NL-00
AA-AE003418NL-00
A.1
A.1
ASSEMBLY SIDE FRAME OUTER LEFT
ASSEMBLY SIDE FRAME OUTER LEFT
1
1
EA EACH
EA EACH
Buy
Buy
In Work
In Work
Assembly
Assembly
A BIW
A BIW
A E Side Structure
A E Side Structure
EC001_02_423_000
EC001_02_423_000
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
N
N
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
NaT
NaT
nan
nan
nan
nan
nan
nan
nan
nan
nan
na

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
NaT
NaT
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
NaT
NaT
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
1|2|1|1|1|11|2
1|2|1|1|1|11|2
AA-AE003465NL-00
AA-AE003465NL-00
A.1
A.1
HOLDER BRACKET GAS STRUT
HOLDER BRACKET GAS STRUT
1
1
EA EACH
EA EACH
Buy
Buy
In Work
In Work
nan
nan
A BIW
A BIW
A E Side Structure
A E Side Structure
EC001_02_423_217
EC001_02_423_217
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
N
N
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
na

Lucky-Harvest
Y
Y
nan
nan
Selina Song
Selina Song
LCN003@luckyharvest.cn
LCN003@luckyharvest.cn
nan
nan
Changshu
Changshu
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Li Dayong
Li Dayong
nan
nan
Y
Y
nan
nan
22 weeks
22 weeks
2018-09-15 00:00:00
2018-09-15 00:00:00
2018-01-05 00:00:00
2018-01-05 00:00:00
2019-02-28 00:00:00
2019-02-28 00:00:00
 
 
 
 
 
 
2019-03-18 00:00:00
2019-03-18 00:00:00
2019-05-20 00:00:00
2019-05-20 00:00:00
2019-07-15 00:00:00
2019-07-15 00:00:00
2019-10-01 00:00:00
2019-10-01 00:00:00
 
 
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Latest revision
Latest revision
Active
Active
1|2|1|1|2|10
1|2|1|1|2|10
AA-AE003421NR-FN
AA-AE003421NR-FN
A.1
A.1
FN-HOLDER FENDER
FN-HOLDER FENDER
1
1
EA EACH
EA EACH
Buy
Buy
In Work
In Work
nan
nan
A BIW
A BIW
A E Side Structure
A E Side Stru

nan
nan
1|2|1|2|1|1|1|1|1
1|2|1|2|1|1|1|1|1
AA-AE003472RN-00
AA-AE003472RN-00
A.1
A.1
REINFORCEMENT LATCH
REINFORCEMENT LATCH
1
1
EA EACH
EA EACH
Buy
Buy
In Work
In Work
nan
nan
A BIW
A BIW
A E Side Structure
A E Side Structure
EC001_02_343_143
EC001_02_343_143
16
16
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
N
N
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
NaT
NaT
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
NaT
NaT
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
1|2|1|2|1|1|1|1|2
1|2|1|2|1|1|1|1|2
AA-AE003447RN-00
AA-AE003447RN-00
A.1
A.1
HOLDER STRIKER TAILGATE
HOLDER STRIKER TAILGATE
1
1
EA EACH
EA EACH
Buy
Buy
In Work
In Work
nan
nan
A BIW
A BIW
A E Side Structure
A E Sid

...

Active
Active
3|4|17|1
3|4|17|1
AA-CF003078FL-00
AA-CF003078FL-00
A.1
A.1
DOOR CLADDING FRT LH
DOOR CLADDING FRT LH
1
1
nan
nan
Buy
Buy
In Work
In Work
nan
nan
C Exterior
C Exterior
C F Trim
C F Trim
EC001_04_501_211
EC001_04_501_211
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
N
N
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
NaT
NaT
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
NaT
NaT
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
3|4|17|2
3|4|17|2
AA-CF003076FL-00
AA-CF003076FL-00
A.1
A.1
DOOR CLAD FINISHER FRT LH
DOOR CLAD FINISHER FRT LH
1
1
nan
nan
Buy
Buy
In Work
In Work
nan
nan
C Exterior
C Exterior
C F Trim
C F Trim
EC001_04_501_221
EC001_04_501_221
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
NaT
NaT
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
NaT
NaT
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
3|4|25
3|4|25
AA-CF002183FL-FN
AA-CF002183FL-FN
A.1
A.1
FASTENERS COVER FENDER LH
FASTENERS COVER FENDER LH
1
1
nan
nan
nan
nan
Released
Released
nan
nan
C Exterior
C Exterior
C F Trim
C F Trim
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
N
N
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
NaT
NaT
nan
nan
nan
na

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
NaT
NaT
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
NaT
NaT
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
3|5|1|3
3|5|1|3
AA-CB002884NN-00
AA-CB002884NN-00
A.1
A.1
Bumper Fascia Diffuser Rear\n
Bumper Fascia Diffuser Rear\n
1
1
nan
nan
Buy
Buy
In Work
In Work
nan
nan
C Exterior
C Exterior
C B Rear Bumper
C B Rear Bumper
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
N
N
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
NaT
NaT
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
Latest revision
Latest revision
Active
Active
3|5|4
3|5|4
AA-CB002874NN-00
AA-CB002874NN-00
A.1
A.1
Stiffener Basic
Stiffener Basic
1
1
EA EACH
EA EACH
Buy
Buy
In Work
In Work
Part
Part
C Exterior
C Exterior
C B Rear Bumper
C B Rear Bumper
EC001_04_201_025
EC001_04_201_025
nan
nan
nan
nan
nan
nan
Mark Bruderick
Mark Bruderick
Eric.Liu
Eric.Liu
Harvey.Li
Harvey.Li
YFPO
YFPO
Y
Y
nan
nan
Huang Liang
Huang Liang
lhuang10@yfpo.com
lhuang10@yfpo.com
nan
nan
Nanjing
Nanjing
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Li.Guo
Li.Guo
nan
nan
Y
Y
nan
nan
 
 
NaT
NaT
nan
nan
nan
nan
 
 
 
 
 
 
NaT
NaT
nan
nan
nan
nan
nan
nan
 
 
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Latest revision
Latest revision
Active
Active
3|5|5
3|5|5
AA-CB002873NN-00
AA-CB002873NN-00
A.1
A.1
BUMPER_BEA

KeyboardInterrupt: 

In [359]:
SCBOM_updated.shape

(4643, 79)